# Let's Pull some Data

I created this notebook so that I can isloate my SQL queuries from the rest of my code. 

In [2]:
# type the following to install the required packages into terminal
# conda activate civil_unrest
# gcloud init
# gcloud auth application-default login

# Some other packages that may be needed
# %pip install db-dtypes


In [3]:
# Import the necessary module
from google.cloud import bigquery
import pandas as pd
from matplotlib import pyplot as plt
import folium
from folium.plugins import HeatMap
import numpy as np
from matplotlib import colors
from geopy.distance import geodesic


In [4]:
# Initialize BigQuery client with the project ID
client = bigquery.Client(project="civil-unrest-predictor")

query = """
SELECT
    SQLDATE,
    EventCode,
    ActionGeo_FullName,
    ActionGeo_Lat,
    ActionGeo_Long,
    AvgTone
FROM
    `gdelt-bq.full.events`
WHERE
    EventCode IN ('145', '1451', '1452', '1453', '1454')
    AND ActionGeo_FullName LIKE '%Chicago%'
    AND CAST(SQLDATE AS STRING) >= '20150101'
ORDER BY
    SQLDATE DESC;
"""

# Execute the updated query
query_job = client.query(query)

# Convert results to a DataFrame
data = query_job.result().to_dataframe()

In [5]:
data

,SQLDATE,EventCode,ActionGeo_FullName,ActionGeo_Lat,ActionGeo_Long,AvgTone
0,20241211,145,"Chicago, Illinois, United States",41.8500,-87.6501,-4.147465
1,20241211,145,"Chicago, Illinois, United States",41.8500,-87.6501,-4.147465
2,20241206,145,"Chicago, Illinois, United States",41.8500,-87.6501,1.066667
3,20241202,145,"University Of Chicago, Illinois, United States",41.7898,-87.5998,-0.843289
4,20241202,145,"Chicago, Illinois, United States",41.8500,-87.6501,-7.840083
...,...,...,...,...,...,...
3344,20150119,145,"Chicago, Illinois, United States",41.8500,-87.6501,2.494577
3345,20150119,145,"Chicago, Illinois, United States",41.8500,-87.6501,2.494577
3346,20150119,145,"Chicago, Illinois, United States",41.8500,-87.6501,2.494577
3347,20150119,145,"Chicago, Illinois, United States",41.8500,-87.6501,2.857143


In [7]:
data.to_csv('data.csv', index=False)